In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Import all the required packages

In [ ]:
import h2o
from h2o.automl import H2OAutoML
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import re
import nltk
from nltk.stem.porter import PorterStemmer
from sklearn.model_selection import train_test_split
from nltk import word_tokenize

Intialize a H2o instance

In [ ]:
h2o.init()

In [ ]:
nltk.download('punkt')

In [ ]:
train = pd.read_csv('/kaggle/input/nlp-getting-started/train.csv')
test = pd.read_csv('/kaggle/input/nlp-getting-started/test.csv')
train

In [ ]:
train['target'].value_counts()

In [ ]:
X_train, X_test = train_test_split(train,test_size=0.2,random_state=111)

Stemming ans stop words

In [ ]:
stemmer = nltk.stem.SnowballStemmer('english')

nltk.download('stopwords')
stop_words = set(nltk.corpus.stopwords.words('english'))

Custom tokenization function

In [ ]:
def tokenize(text):
    tokens = [word for word in word_tokenize(text) if word.isalpha()]
    tokens = map(str.lower,tokens)
    stems = [stemmer.stem(item) for item in tokens if (item not in stop_words)]
    
    return stems

In [ ]:
 vectorizer_tf = TfidfVectorizer(tokenizer=tokenize,stop_words=None, max_df=0.75, max_features=1000, lowercase=False, ngram_range=(1,2))
 train_vectors = vectorizer_tf.fit_transform(X_train.text)

In [ ]:
test_vectors = vectorizer_tf.transform(X_test.text)

In [ ]:
train_df = pd.DataFrame(train_vectors.toarray(),columns=vectorizer_tf.get_feature_names())
train_df = pd.concat([train_df,X_train['target'].reset_index(drop=True)],axis=1)
train_df

In [ ]:
test_df = pd.DataFrame(test_vectors.toarray(),columns=vectorizer_tf.get_feature_names())
test_df = pd.concat([test_df,X_test['target'].reset_index(drop=True)],axis=1)
test_df

Convert data frame to H2o object

In [ ]:
h2o_train_df = h2o.H2OFrame(train_df)
h2o_test_df = h2o.H2OFrame(test_df)

In [ ]:
h2o_train_df['target'] = h2o_train_df['target'].asfactor()
h2o_test_df['target'] = h2o_test_df['target'].asfactor()

Define AutoML. Here we can change thw parameters to get better results

In [ ]:
aml = H2OAutoML(max_models=10,seed=10,exclude_algos=['StackedEnsemble'],verbosity="info",nfolds=0,balance_classes=True,max_after_balance_size=0.3)

In [ ]:
x = vectorizer_tf.get_feature_names()
y = 'target'

In [ ]:
aml.train(x=x, y=y, training_frame=h2o_train_df, validation_frame=h2o_test_df)

In [ ]:
aml.leaderboard

In [ ]:
pred = aml.leader.predict(h2o_test_df)

In [ ]:
aml.leader.model_performance(h2o_test_df)

In [ ]:
model_ids = list(aml.leaderboard['model_id'].as_data_frame().iloc[:,0])
out = h2o.get_model([mid for mid in model_ids if "GLM" in mid][0])

In [ ]:
out